# Stockage de données dans un fichier externe.
## Besoin
Dans le cadre du projet anti-tabac du lycée, on est amené à définir un dictionnaire associant un son à jouer à une ville parcourue, représentée par un kilométrage.
Dans le programme, on définit donc un dictionnaire affectant à un kilométrage un son.
Ainsi, on retrouve dans le programme la définition de dictionnaire suivante :

In [1]:
CITIES = {
    59500: 'sounds/guingamp.wav',
    92400: 'sounds/saintBrieuc.wav',
    213000: 'sounds/rennes.wav'
    # etc...
}

Cependant, il est possible que l'on souhaite modifier ce dictionnaire : en l'état actuel des choses, une modification manuelle du code source est nécessaire. On souhaite donc que l'utilisateur n'ait pas à modifier le code source, et on veut donc externaliser le stockage de données, dans un fichier en dehors du code source. Nous allons étudier deux solutions qui permettent de stocker des données dans un fichier externe : le JSON et le CSV.

## Préalable : lecture d'un fichier externe en Python
La lecture d'un fichier externe en Python se fait *via* la fonction `open` de Python : elle prend en paramètre une chaîne de caractère qui contient l'emplacement du fichier sur la machine, et qui renvoie un objet de type `TextIOWrapper`.
Cependant, il faut éviter d'utiliser la fonction seule : en effet, un plantage du programme alors que le fichier est ouvert pourrait empêcher la fermeture correcte du fichier, et poser un problème en cas de réouverture du fichier par le programme.
Pour permettre la fermeture du fichier en cas de problème, on utilisera le mot clé `with`.

Par exemple, si l'on veut ouvrir un fichier et stocker son `TextIOWrapper` dans la variable `file` on utilisera la syntaxe :
````python
with open('nomFichier') as file:
    # Manipulations sur le fichier.
````

En cas d'erreur dans le nom du fichier, une exception `FileNotFoundError` sera levée.

Pour lire le contenu du fichier file, on utilisera la fonction `TextIOWrapper.read`, qui renverra une chaîne de caractère.
Enfin, on fermera le fichier avec la fonction `TextIOWrapper.close`.

En application, avec le fichier `files/text.txt`:

In [2]:
with open('files/text.txt') as file: # Ouverture et gestion sécurisée du fichier.
    text = file.read() # Lecture du fichier.
    print('Contenu du fichier :', text) # Affichage du contenu du fichier.
    file.close() # Fermeture du fichier.

FileNotFoundError: [Errno 2] No such file or directory: 'files/text.txt'

## Première solution : le JSON
### Définition
Le JSON (JavaScript Object Notation) est un langage de définition de données, basé sur les objets Javascript. Il permet la définition de données en les plaçant dans des blocs imbriqués les uns dans les autres.
### Structure de base
En premier lieu, l'ensemble des données stockées dans un fichier JSON doivent être dans un `object` : qui se définit avec des accolades, comme qui suit :

On doit ensuite définir les données sous forme de propriétés Javascript, en suivant le modèle suivant :

`"nomPropriété"` est obligatoirement une chaîne de caractère (entre guillemets `"`), `<valeur>` est une valeur pouvant prendre les différents types de données Javascript (booléen, nombre, chaîne de caractère, liste, objet, null...). La virgule en fin de propriété ne doit être mise seulement s'il y a une propriété après.

On a donc par exemple :

Il est même possible d'imbriquer des objets, ou d'utiliser des listes pour stocker plusieurs valeurs dans une seule propriété :

### Comment lire du JSON en Python ?
Pour lire du JSON en Python, il existe un module de la bibliothèque standard Python nommé `json` :

In [3]:
import json

Pour lire du JSON en Python à partir d'une chaîne de caractère, on utilise la fonction `json.loads` qui prend en paramètre une chaîne de caractère JSON et qui renvoie un dictionnaire correspondant au contenu de la chaîne JSON :

In [4]:
json.loads('{"prop1": 10.5, "prop2": "test", "propN": true}')

{'prop1': 10.5, 'prop2': 'test', 'propN': True}

En cas d'erreur dans le code, une exception `JSONDecodeError` est levée :

In [5]:
json.loads('{Chaîne json invalide: 10.5}')

JSONDecodeError: Expecting property name enclosed in double quotes: line 1 column 2 (char 1)

Pour lire depuis un fichier extérieur, on applique la méthode précédemment vue :

In [6]:
with open('files/test.json') as jsonFile: # Ouverture fichier.
    fileContent = jsonFile.read() # Lecture fichier.
    datas = json.loads(fileContent) # Lecture JSON.
    print('Données :\n{}'.format(datas)) # Affichage dictionnaire issu du JSON.
    jsonFile.close() # Fermeture fichier.

Données :
{'prop1': 10.5, 'prop2': 'test', 'propN': True}


### Application au projet
On peut stocker les données du dictionnaire suivant une syntaxe JSON. On peut, à partir du dictionnaire au début du notebook, noter :

On stockera les données dans le fichier [`files/dict.json`](files/dict.json).
On stockera le dictionnaire obtenu après lecture du fichier dans une variable `jsonData`.

In [7]:
jsonData = None

On lit tout d'abord le fichier :

In [8]:
with open('files/dict.json') as file:
    jsonData = json.loads(file.read()) # Lecture des données JSON du fichier.
    file.close()

On obtient donc :

In [9]:
jsonData

{'59500': 'sounds/guingamp.wav',
 '92400': 'sounds/saintBrieuc.wav',
 '213000': 'sounds/rennes.wav'}

On remarque que les distance sont de type `str` (chaîne de caractère), alors que les distance du dictionnaire prévu de base sont de type `float`.
Pour résoudre le problème, on a deux possibilités :
1. Utiliser le dictionnaire récupéré depuis le fichier JSON comme dictionnaire du programme, puis convertir la distance récupérée depuis ThingSpeak (type `float`) en chaîne de caractère (type `str`) avec la méthode `str`. On définirait donc le dictionnaire du programme par : 
````python
CITIES = jsonData
````
2. Convertir les distances du dictionnaire JSON du type `str` au type `float`, avec la méthode `float`. Il faudrait alors lire les clés du dictionnaire jsonData, les convertir en `float` et ajouter la valeur correspondante à la clé convertie en `float` sur le dictionnaire `CITIES`, on aura donc :
````python
for k in jsonData.keys(): # On parcourt une à une les clés du dictionnaire jsonData.
        CITIES[float(k)] = jsonData[k] # On convertit la clé en float pour le dictionnaire CITIES, et on attribue la valeur de jsonData correspondante.
````

## Deuxième solution : le CSV
Le format JSON, bien qu'apprécié, nécessite cependant des connaissances de base en programmation et en description de données. On souhaite faciliter l'entrée de données, et utiliser éventuellement un tableur, qui est un outil plus intuitif. Or les tableurs ne peuvent pas exporter de données JSON, à moins d'installer une extension. Ils peuvent cependant exporter sous un autre format de données textuel, le CSV.
### Qu'est-ce que le CSV ?
CSV signifie &laquo;*Comma-separated values*&raquo;. Il s'agit d'un format libre de description de données. Moins utilisé, il est cependant disponible à l'enregistrement/exportation dans de nombreux tableurs.

![Options d'exportation en CSV dans différents tableurs](img/tableurs_csv.png)

Le CSV permet de définir des séries de listes de données, à partir de données rentrées sur tableur. Dans la syntaxe originale, une rangée de tableur contenant les valeurs &laquo;a&raquo;, &laquo;b&raquo;, &laquo;c&raquo; sera notée :

Si on rajoute une rangée avec les valeurs 10.5, 58, &laquo;test2&raquo;, on a :

On peut avoir, suivant les configurations d'exportation, des séparateurs pour repérer les chaînes de caractères, comme des guillemets `"`, on a donc :

#### Norme française
Le CSV, contrairement au JSON, n'est pas régit par une norme internationale, il peut donc y avoir des variations.
La plus notable concerne la notation des décimales : en notation anglaise, on écrira par exemple `1.5`, alors qu'en notation française, on écrira `1,5`.
Cette variation est visible dans les tableurs, ainsi, en France, on utilisera sur un tableur la virgule `,`. Cela a pour conséquence de modifier les séparateurs CSV : pour éviter la confusion entre les décimales et les séparations de valeurs, on utilisera le séparateur point-virgule `;` en France :

### Comment lire du CSV en Python ?
Le CSV peut être lu en Python avec le module `csv` de la bibliothèque standard :

In [10]:
import csv

On prendra ici l'exemple de la feuille de calcul suivante :
![Feuille de calcul](img/excel.png)
On a donc en CSV, dans le fichier [`files/test.csv`](files/test.csv) :

On indique d'abord, dans des variables, quels séparateurs on utilise :

In [11]:
VAL_SEPARATOR = ';' # Séparateur entre les valeurs.
IS_QUOTING_ENABLED = False # Y a-t-il des chaînes de caractères ?
STR_SEPARATOR = '' # Séparateur de chaîne de caractère.

Pour permettre la lecture on doit tout d'abord ouvrir le fichier, que l'on stockera dans une variable `csvFile`. On utilisera ensuite la fonction `csv.reader`, qui prend en paramètre le fichier ainsi que les séparateurs utilisés.
S'il y a des chaînes de caractères, on donne le séparateur de chaîne de caractère, sinon on ne le définit pas.
````python
csvRawData = csv.reader(
    csvFile, 
    delimiter=VAL_SEPARATOR, 
    quotechar=STR_SEPARATOR if IS_QUOTING_ENABLED else None
)
````
La valeur de retour de `csv.reader` est un objet de type `_csv.reader` : pour faciliter son exploitation, on le convertit en liste :
````python
csvData = list(csvRawData)
````
On a donc :

In [12]:
csvData = None
with open('files/test.csv') as csvFile: # Ouverture fichier
    csvRawData = csv.reader(
        csvFile, 
        delimiter=VAL_SEPARATOR, 
        quotechar=STR_SEPARATOR if IS_QUOTING_ENABLED else None # Modification du caractère de séparation des chaînes si il y a des chaînes.
    ) # Lecture données CSV
    csvData = list(csvRawData) # Conversion en liste
    csvFile.close() # Fermeture du fichier.

On obtient la liste suivante :

In [13]:
csvData

[['prop1', '10,5', ''], ['prop2', 'test'], ['propN', 'true']]

On la parcourt et on affiche les valeurs :

In [14]:
for c in csvData: # Rangées
    for c1 in c: # Colonnes
        print(c1, end=' ')
    print('\n') # Retour à la ligne.

prop1 10,5  

prop2 test 

propN true 



### Application au projet

On peut définir les données du dictionnaire sur tableur comme qui suit :
![Feuille de calcul dictionnaire](img/excel_dico.png)

Après exportation en CSV, on obtient le fichier [`files/dico.csv`](files/dico.csv), ayant le contenu suivant :

On utilisera les mêmes séparateurs que dans les exemples précédents :
````python
VAL_SEPARATOR = ';'
IS_QUOTING_ENABLED = False
STR_SEPARATOR = ''
````

On récupère les données du fichier :

In [15]:
csvData = None
with open('files/dico.csv') as csvFile: # Ouverture fichier
    csvRawData = csv.reader(
        csvFile, 
        delimiter=VAL_SEPARATOR, 
        quotechar=STR_SEPARATOR if IS_QUOTING_ENABLED else None # Modification du caractère de séparation des chaînes si il y a des chaînes.
    ) # Lecture données CSV
    csvData = list(csvRawData) # Conversion en liste
    csvFile.close() # Fermeture du fichier.

On obtient la liste suivante :

In [16]:
csvData

[['59500,0', 'sounds/guingamp.wav'],
 ['92400,0', 'sounds/saintBrieuc.wav'],
 ['213000,0', 'sounds/rennes.wav']]

On remarque, parce que les valeurs ont été entrées sur un tableur configuré en saisie française, que les valeurs décimales possèdent une virgule `,` au lieu d'un point `.` : on risque de se retrouver, lors d'une conversion en nombre, avec une `ValueError`, ou bien d'avoir des problèmes de comparaisons. 

In [17]:
float('59500,0')

ValueError: could not convert string to float: '59500,0'

On doit donc remplacer la virgule `,` en un point `.`.
On utilisera la fonction `str.replace`, qui permet de remplacer une chaîne par un autre chaîne dans la chaîne sur laquelle elle est appelée: elle prendra en paramètre la chaîne à changer (type `str`) et la chaîne remplacante (type `str`), et renverra la chaîne ainsi traitée.

On parcourt donc la liste, et on change l'élément d'index 0 de chaque entrée de donnée (la distance), afin d'obtenir des nombres valides :

In [18]:
for i in csvData: # On parcourt les éléments de csvData
    i[0] = i[0].replace(',', '.') # On remplace les virgules par des points.

On obtient :

In [19]:
csvData

[['59500.0', 'sounds/guingamp.wav'],
 ['92400.0', 'sounds/saintBrieuc.wav'],
 ['213000.0', 'sounds/rennes.wav']]

Par la suite on a, comme dans le cas du JSON, la possiblité de convertir les valeurs de distance obtenues *via* ThingSpeak grâce à la fonction `str`, ou de convertir les valeurs de `csvData` en nombre avec la fonction `float`. Dans les deux cas, le dictionnaire devra être construit en parcourant la liste `csvData` :

**Premier cas :**

In [20]:
CITIES = {} # Réinitialisation dictionnaire
for c in csvData: # On parcourt csvData.
    CITIES[c[0]] = c[1] # On associe c[1] (son à jouer) à c[0] distance parcourue.

On obtient :

In [21]:
CITIES

{'59500.0': 'sounds/guingamp.wav',
 '92400.0': 'sounds/saintBrieuc.wav',
 '213000.0': 'sounds/rennes.wav'}

**Deuxième cas :**

In [22]:
CITIES = {} # Réinitialisation dictionnaire
for c in csvData:
    CITIES[float(c[0])] = c[1] # Même procédure, mais on convertit en float c[0].

On obtient :

In [23]:
CITIES

{59500.0: 'sounds/guingamp.wav',
 92400.0: 'sounds/saintBrieuc.wav',
 213000.0: 'sounds/rennes.wav'}